# Revista Catarinense da Ciência Contábil

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h3', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'editorial' in item.find('a').text.lower()
            remover_links = remover_links or 'folha de rosto' in item.find('a').text.lower()
            remover_links = remover_links or 'dados da edição ' in item.find('a').text.lower()
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://revista.crcsc.org.br/index.php/CRCSC/search/index?query=__query__&searchJournal=15&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://revista.crcsc.org.br/index.php/CRCSC/article/view/1259'

In [6]:
links

['https://revista.crcsc.org.br/index.php/CRCSC/article/view/1259',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2263',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/3168',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2315',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2508',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2402',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/1224',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2153',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2086',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2753',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/1257',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/3019',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/3148',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/2170',
 'https://revista.crcsc.org.br/index.php/CRCSC/article/view/31

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('–')[0]
    afiliacao = afiliacao.split(' - ')[0]
    afiliacao = afiliacao.split('(')[0]
    afiliacao = afiliacao.split('/')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'Sustentabilidade Ambiental em Condomínios: utilização do método SICOGEA para avaliar os aspectos e impactos ambientais em um condomínio residencial - DOI: http://dx.doi.org/10.16930/2237-7662/rccc.v11n31p72-83'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '23/05/2012'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'Este estudo teve por objetivo avaliar os aspectos e impactos ambientais de um condomínio residencial com aporte da Contabilidade Gerencial Ambiental. Isto se deu mediante a identificação das situações deficitárias de maior prioridade para o condomínio, bem como da análise do grau de sustentabilidade obtido por meio da aplicação parcial do método SICOGEA - Sistema Contábil Gerencial Ambiental. O método utilizado foi o estudo de caso, com pesquisa exploratório-descritiva, envolvendo entrevista semiestruturada com a síndica deste condomínio. Analisando os resultados obtidos, com base no SICOGEA, constatou-se um grau de sustentabilidade global de 52%, considerado regular. Isto expressou, também, uma situação de desempenho ambiental médio, atendendo à legislação. Não obstante, pode-se considerar que o desempenho ambiental desta organização está aquém da almejada valorização ambiental e prevenção da poluição. Dos cinco critérios pesquisados, três apresentaram sustentabilidade def

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Sustentabilidade. Gestão ambiental. SICOGEA. Condomínio residencial.'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Jerusa Bacelo',
 'Autor 2': 'Vivian Osmari Uhlmann',
 'Autor 3': 'Elisete Dahmer Pfitscher',
 'Autor 4': 'Maíra Melo de Souza',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal de Santa Catarina',
 'Afiliação 2': 'Universidade Federal de Santa Catarina',
 'Afiliação 3': 'Universidade Federal de Santa Catarina',
 'Afiliação 4': 'Universidade Federal de Santa Catarina',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista Catarinense da Ciência Contábil'}
    url_search_text = 'https://revista.crcsc.org.br/index.php/CRCSC/search/index?query=__query__&searchJournal=15&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'url': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%d/%m/%Y')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,url,Revista
0,2012-05-23,Sustentabilidade Ambiental em Condomínios: uti...,Este estudo teve por objetivo avaliar os aspec...,Sustentabilidade. Gestão ambiental. SICOGEA. C...,4,Jerusa Bacelo,Vivian Osmari Uhlmann,Elisete Dahmer Pfitscher,Maíra Melo de Souza,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
1,2017-04-28,A Teoria Institucional aplicada à Contabilidad...,O objetivo da presente pesquisa é analisar as ...,Teoria Institucional. Contabilidade. Contabili...,3,José Luiz Vailatti,Fabricia da silva Rosa,Ernesto Fernando Rodrigues Vicente,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
2,2021-09-30,Influência do clima ético na conduta de trapaç...,A trapaça acadêmica mostrou-se uma preocupação...,Ensino Contábil. Estudantes de Contabilidade. ...,3,Igor Pereira da Luz,Pamela Raquel Wagnitz,Rodrigo Rengel,,,,Universidade Federal de Santa Catarina,Centro Universitário Estácio de Sá,Universidade Federal de Santa Catarina,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil


In [24]:
df.to_csv('../data/RCCC.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,url,Revista
0,2012-05-23,Sustentabilidade Ambiental em Condomínios: uti...,Este estudo teve por objetivo avaliar os aspec...,Sustentabilidade. Gestão ambiental. SICOGEA. C...,4,Jerusa Bacelo,Vivian Osmari Uhlmann,Elisete Dahmer Pfitscher,Maíra Melo de Souza,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
1,2017-04-28,A Teoria Institucional aplicada à Contabilidad...,O objetivo da presente pesquisa é analisar as ...,Teoria Institucional. Contabilidade. Contabili...,3,José Luiz Vailatti,Fabricia da silva Rosa,Ernesto Fernando Rodrigues Vicente,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
2,2021-09-30,Influência do clima ético na conduta de trapaç...,A trapaça acadêmica mostrou-se uma preocupação...,Ensino Contábil. Estudantes de Contabilidade. ...,3,Igor Pereira da Luz,Pamela Raquel Wagnitz,Rodrigo Rengel,,,,Universidade Federal de Santa Catarina,Centro Universitário Estácio de Sá,Universidade Federal de Santa Catarina,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
3,2017-08-22,Influência dos investimentos ambientais e dos ...,O objetivo geral do trabalho é verificar a inf...,Índice de Sustentabilidade Empresarial (ISE). ...,3,Barbara Beirão Tonolli,Suliani Rover,Denize Demarche Minatti Ferreira,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
4,2018-08-08,Incentivos fiscais como política pública de de...,"No Brasil, a competição tributária entre os go...",Incentivos fiscais. guerra fiscal. desenvolvim...,4,Julio Cesar Fazoli,Fabrícia da Silva Rosa,Leonardo Flach,Luiz Felipe Ferreira,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
5,2017-08-22,Índice de Disclosure Ambiental (IDA): análise ...,As empresas preocupam-se em divulgar informaçõ...,Índice de Disclosure Ambiental (IDA). Relatóri...,3,Stella Maris Lima Altoé,Luiz Panhoca,Márcia Maria dos Santos Bortolocci Espejo,,,,Universidade Estadual do Centro-Oeste,Universidade Federal do Paraná,Universidade Federal do Mato Grosso do Sul,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
6,2012-01-23,Desempenho econômico e evidenciação ambiental:...,Nas últimas décadas cresce o interesse da soci...,Evidenciação ambiental. Desempenho econômico. ...,3,Gislaine Pereira,Fernando Nitz de Carvalho,Edna Ghiorzi Varela Parente,,,,,,,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
7,2015-12-10,A Relação entre o Ingresso ou Saída de Empresa...,A preocupação ambiental e social nas ações hum...,Sustentabilidade. Rentabilidade. ICO2,3,Vinicius Mothé Maia,Filipe Pollis de Carvalho,Liege Moraes do Carmo,,,,FACC,IAG,FACC,,,,https://revista.crcsc.org.br/index.php/CRCSC/a...,Revista Catarinense da Ciência Contábil
8,2015-08-25,Responsabilidade social e retornos das ações: ...,A crescente preocupação com a questão social e...,Responsabilidade social corporativa. Índice de...,3,Diego Lopes de Oliveira Martins,Valéria Gama Fully Bressan,Renata Turola Takamatsu,,,,Universidade Federal de Minas Gerais,Universidade Federal de Minas Gerais,Universidade Federal de Minas Gerai